<a href="https://colab.research.google.com/github/BaranGursoy/RetinaNet/blob/main/centernetbetter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FateScript/CenterNet-better

Cloning into 'CenterNet-better'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 154 (delta 3), reused 5 (delta 1), pack-reused 142
Receiving objects: 100% (154/154), 191.70 KiB | 14.75 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 18.3MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=535735794ae860b4cc42430670c0dc860f1a962a1fa3b0305fc1c2afb4213cc3
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 244kB/s 
     |████████████████████████████████| 7.6MB 13.1MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1+cu101 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101


In [1]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")

Obtain Dataset

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/VisDrone2019-DET-train.zip -d /content/

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000140.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000141.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000142.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000143.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000144.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000145.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000146.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000147.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000148.jpg  
 extracting: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000149.jpg  
  inflating: /content/VisDrone2019-DET-train/images/9999937_00000_d_0000150.jpg  
  inflating: /content/VisDrone2019-DET-train/images

In [4]:
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-poksxk90
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-poksxk90
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263916 sha256=c1293f721ea2c4953058df1585eec222cb2a77734ad7f66bd8fed0102cc4796f
  Stored in directory: /tmp/pip-ephem-wheel-cache-3r_ywvw0/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [5]:
import os
%cd /content/CenterNet-better
!pip install -e .

/content/CenterNet-better
Obtaining file:///content/CenterNet-better
  Running setup.py develop for dl-lib


In [18]:
%cd /content/CenterNet-better/playground/centernet.res101.coco.512size

/content/CenterNet-better/playground/centernet.res101.coco.512size


In [6]:
!pip install colorama

In [8]:
import cv2
import os
import numpy as np


input_img_folder = '/content/VisDrone2019-DET-train/images'
input_ann_folder = '/content/VisDrone2019-DET-train/annotations'
output_ann_folder = '/content/VisDrone2019-DET-train/annotations_new'
output_img_folder = '/content/VisDrone2019-DET-train/images_new'

os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(output_ann_folder, exist_ok=True)


image_list = os.listdir(input_img_folder)
annotation_list = os.listdir(input_ann_folder)

label_dict = {
	"0" : "Ignore",
	"1" : "Pedestrian",
	"2" : "People",
	"3" : "Bicycle",
	"4" : "Car",
	"5" : "Van",
	"6" : "Truck",
	"7" : "Tricycle",
	"8" : "Awning-tricycle",
	"9" : "Bus",
	"10" : "Motor",
	"11" : "Others"
}

thickness = 2
color = (255,0,0)
count = 0

def object_string(label, bbox):
	req_str = '''
	<object>
		<name>{}</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>{}</xmin>
			<ymin>{}</ymin>
			<xmax>{}</xmax>
			<ymax>{}</ymax>
		</bndbox>
	</object>
	'''.format(label, bbox[0], bbox[1], bbox[2], bbox[3])
	return req_str

for annotation in annotation_list:
	annotation_path = os.path.join(os.getcwd(), input_ann_folder, annotation)
	xml_annotation = annotation.split('.txt')[0] + '.xml'
	xml_path = os.path.join(os.getcwd(), output_ann_folder, xml_annotation)
	img_file = annotation.split('.txt')[0] + '.jpg'
	img_path = os.path.join(os.getcwd(), input_img_folder, img_file)
	output_img_path = os.path.join(os.getcwd(), output_img_folder, img_file)
	img = cv2.imread(img_path)
	annotation_string_init = '''
<annotation>
	<folder>annotations</folder>
	<filename>{}</filename>
	<path>{}</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>{}</width>
		<height>{}</height>
		<depth>{}</depth>
	</size>
	<segmented>0</segmented>'''.format(img_file, img_path, img.shape[0], img.shape[1], img.shape[2])

	file = open(annotation_path, 'r')
	lines = file.readlines()
	for line in lines:
		new_line = line.strip('\n').split(',')
		new_coords_min = (int(new_line[0]), int(new_line[1]))
		new_coords_max = (int(new_line[0])+int(new_line[2]), int(new_line[1])+int(new_line[3]))
		bbox = (int(new_line[0]), int(new_line[1]), int(new_line[0])+int(new_line[2]), int(new_line[1])+int(new_line[3]))
		label = label_dict.get(new_line[5])
		req_str = object_string(label, bbox)
		annotation_string_init = annotation_string_init + req_str
		#cv2.rectangle(img, new_coords_min, new_coords_max, color, thickness)
	cv2.imwrite(output_img_path, img)
	annotation_string_final = annotation_string_init + '</annotation>'
	f = open(xml_path, 'w')
	f.write(annotation_string_final)
	f.close()
	count += 1
	print('[INFO] Completed {} image(s) and annotation(s) pair'.format(count))

Görüntülenen çıkış son 5000 satıra kısaltıldı.
[INFO] Completed 1472 image(s) and annotation(s) pair
[INFO] Completed 1473 image(s) and annotation(s) pair
[INFO] Completed 1474 image(s) and annotation(s) pair
[INFO] Completed 1475 image(s) and annotation(s) pair
[INFO] Completed 1476 image(s) and annotation(s) pair
[INFO] Completed 1477 image(s) and annotation(s) pair
[INFO] Completed 1478 image(s) and annotation(s) pair
[INFO] Completed 1479 image(s) and annotation(s) pair
[INFO] Completed 1480 image(s) and annotation(s) pair
[INFO] Completed 1481 image(s) and annotation(s) pair
[INFO] Completed 1482 image(s) and annotation(s) pair
[INFO] Completed 1483 image(s) and annotation(s) pair
[INFO] Completed 1484 image(s) and annotation(s) pair
[INFO] Completed 1485 image(s) and annotation(s) pair
[INFO] Completed 1486 image(s) and annotation(s) pair
[INFO] Completed 1487 image(s) and annotation(s) pair
[INFO] Completed 1488 image(s) and annotation(s) pair
[INFO] Completed 1489 image(s) and 

In [15]:
%cd /content
!python voc2coco.py ./traindata/annotations_new ./traindata/coco.json

/content
Number of xml files: 6471
Success: ./traindata/coco.json


TRAINING PART

In [7]:
import os
counter = 0
img_path = '/content/VisDrone2019-DET-train/images'
annot_path = '/content/VisDrone2019-DET-train/annotations' 

for path, subdirs, files in os.walk(annot_path):
  for filename in files:  
    anot_p = os.path.join(annot_path, filename[:-4] + '.txt')

    os.rename(anot_p, annot_path + '/'+ str(counter) + '.txt')


    counter = counter + 1


counter = 0

for path, subdirs, files in os.walk(img_path):
  for filename in files:  
    img_p = os.path.join(path, filename)

    os.rename(img_p, img_path + '/' + str(counter) + '.jpg')

    counter = counter + 1    

In [ ]:
!dl_train --num-gpus 1

soft link to /data/Outputs/model_logs/playground/centernet.res101.coco.512size
Command Line Args: Namespace(dist_url='tcp://127.0.0.1:49152', eval_only=False, machine_rank=0, num_gpus=8, num_machines=1, opts=[], resume=False)
